In [669]:
import pandas as pd
from copy import copy
import numpy as np
import re

# Task 1

In [670]:
data1 = pd.read_csv('input1.txt', header=None, sep='\n')

sum_incr = data1[0].rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0]).sum().astype(int)
sum_window_incr = data1[0].rolling(3).sum().rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0]).sum().astype(int)

print('ans1: ' + str(sum_incr)) # 1766
print('ans2: ' + str(sum_window_incr)) # 1797

ans1: 1766
ans2: 1797


# Task 2

In [671]:
data2 = pd.read_csv('input2.txt', header=None, sep=' ')

sum_forward = data2[data2[0] == 'forward'][1].sum()
sum_down = data2[data2[0] == 'down'][1].sum()
sum_up = data2[data2[0] == 'up'][1].sum()

aim = 0
depth = 0

for a in data2.iterrows():
    if a[1][0] == 'forward':
        depth += aim * a[1][1]
    elif a[1][0] == 'down':
        aim += a[1][1]
    elif a[1][0] == 'up':
        aim -= a[1][1]

print('ans1: ' + str(sum_forward * (sum_down - sum_up))) # 1488669
print('ans2: ' + str(sum_forward * depth)) # 1176514794

ans1: 1488669
ans2: 1176514794


# Task 3

In [131]:
# 0101 ^ 1111 = 1010
5 ^ 15

10

In [672]:
data3 = pd.read_csv('input3.txt', header=None, sep=' ', dtype=str)

# 3.1
gamma_most_common = ''.join(data3[0].str.split('', expand=True).iloc[:, 1:-1].apply(lambda x: x.value_counts(), axis='index').idxmax().to_numpy())
gamma_rate = int(gamma_most_common, 2)

# 3.2
data3m1 = copy(data3[0].str.split('', expand=True).iloc[:, 1:-1])
data3m2 = copy(data3m1)

i = 0
while data3m1.shape[0] != 1:
    a = data3m1.iloc[:, i]
    a_count = a.value_counts()
    data3m1 = data3m1[a == ('1' if (a_count.shape == 2 and a_count[0] == a_count[1]) else a_count.idxmax())]
    i += 1

oxygen_rating = int(''.join(data3m1.to_numpy().flatten()), 2)

i = 0
while data3m2.shape[0] != 1:
    a = data3m2.iloc[:, i]
    a_count = a.value_counts()
    data3m2 = data3m2[a == ('0' if (a_count.shape == 2 and a_count[0] == a_count[1]) else a_count.idxmin())]
    i += 1

co2_rating = int(''.join(data3m2.to_numpy().flatten()), 2)

print('ans1: ' + str(gamma_rate * (gamma_rate ^ int('1' * len(gamma_most_common), 2)))) # 4191876
print('ans2: ' + str(oxygen_rating * co2_rating)) # 3424642

ans1: 4191876
ans2: 3424642


# Task 4

In [590]:
data4 = pd.read_csv('input4.txt', header=None, sep="\n", dtype=str)
data4h, data4d = data4.loc[0, :], data4.loc[1:, :]

data4d = data4d[0].str.strip().str.split('\s+', expand=True)
data4d1, data4d2 = copy(data4d), copy(data4d)
# display(data4d)

small_index = np.array_split(data4d.index, 500//5)

boards_won = []

res1, res2 = 0, 0

for num in data4h[0].split(','):
    data4d1 = data4d1.replace(num, np.nan)

    for i in range(100):
        if data4d1.loc[small_index[i], :].dropna(how='all', axis=0).dropna(how='all', axis=1).shape != (5,5):
            res1 = int(data4d1.loc[small_index[i], :].astype(float).sum().sum() * int(num))
            break
    
    if res1 != 0:
        break

for num in data4h[0].split(','):
    data4d2 = data4d2.replace(num, np.nan)
    
    for i in range(100):
        if i not in boards_won and data4d2.loc[small_index[i], :].dropna(how='all', axis=0).dropna(how='all', axis=1).shape != (5,5):
            boards_won.append(i)
            
            if len(boards_won) == 100:
                res2 = int(data4d2.loc[small_index[i], :].astype(float).sum().sum() * int(num))
                break
    
    if res2 != 0:
        break
        
print('ans1: ' + str(res1)) # 41503
print('ans2: ' + str(res2)) # 3178

ans1: 41503
ans2: 3178


# Task 5

In [591]:
data5 = pd.read_csv('input5.txt', header=None, sep="\n", dtype=str)
data5m = data5[0].str.split(r'[,(\-> )]', expand=True).drop(columns=[2, 3, 4]).astype(int)
size = data5m.max().max().astype(int)
grid1 = pd.DataFrame(np.zeros((size, size))).astype(int)
grid2 = pd.DataFrame(np.zeros((size, size))).astype(int)
# display(grid2)

for a in data5m.iterrows():
    x1 = a[1][0]
    x2 = a[1][5]
    y1 = a[1][1]
    y2 = a[1][6]
    
    if x1 == x2 or y1 == y2:
        x1, x2 = min(x1, x2), max(x1, x2)
        y1, y2 = min(y1, y2), max(y1, y2)
        
        grid1.loc[x1-1:x2-1, y1-1:y2-1] += 1
        grid2.loc[x1-1:x2-1, y1-1:y2-1] += 1
    else:
        if x2 > x1:
            xr1, xr2, xd = x1, x2+1, 1
        else:
            xr1, xr2, xd = x1, x2-1, -1
            
        if y2 > y1:
            yr1, yr2, yd = y1, y2+1, 1
        else:
            yr1, yr2, yd = y1, y2-1, -1
        
        for a, b in zip(range(xr1, xr2, xd), range(yr1, yr2, yd)):
            grid2.loc[a-1, b-1] += 1
        
print('ans1: ' + str(grid1[grid1 > 1].count().sum())) # 7085
print('ans2: ' + str(grid2[grid2 > 1].count().sum())) # 20271

ans1: 7085
ans2: 20271


# Task 6

In [592]:
data6 = pd.read_csv('input6.txt', header=None, sep="\n", dtype=str)
data6m = list(map(int, data6.loc[0, 0].split(',')))
# print(data6m,)

for day in range(80):
# for day in range(256):
    new_count = data6m.count(0)
    
    data6m = list(map(lambda x: x-1 if x != 0 else 6, data6m))
    if new_count > 0:
        data6m = data6m + ([8]*new_count)

# print(data6m,)
print(len(data6m))
print('ans1: ' + str(len(data6m))) # 377263

377263
ans1: 377263


# Task 9

In [667]:
data9 = pd.read_csv('input9.txt', header=None, sep="\n", dtype=str)
data9m = data9[0].str.split('', expand=True).drop(columns=[0,101])
data9m.columns = list(range(100))
display(data9m)
data9m.applymap(lambda x: x)
# TODO

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,9,8,7,6,5,6,7,8,9,6,...,3,9,9,9,8,8,8,6,6,7
1,5,9,9,5,4,5,6,9,7,6,...,9,8,9,8,7,6,5,4,5,6
2,6,9,7,6,5,7,7,8,9,7,...,9,7,8,9,8,9,4,3,4,7
3,7,9,8,8,6,7,8,9,8,9,...,5,6,7,8,9,3,2,1,2,3
4,8,9,9,8,7,8,9,8,6,7,...,7,8,9,9,6,5,3,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,9,5,6,7,8,9,9,8,9,...,8,9,4,2,9,6,7,9,2,3
96,8,9,8,7,9,9,9,8,7,8,...,9,3,2,1,2,5,9,8,9,4
97,9,4,9,9,9,9,8,7,6,7,...,3,2,1,0,3,4,5,7,8,9
98,5,3,2,3,9,8,7,7,5,6,...,4,4,2,1,2,5,6,8,9,5


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,9,8,7,6,5,6,7,8,9,6,...,3,9,9,9,8,8,8,6,6,7
1,5,9,9,5,4,5,6,9,7,6,...,9,8,9,8,7,6,5,4,5,6
2,6,9,7,6,5,7,7,8,9,7,...,9,7,8,9,8,9,4,3,4,7
3,7,9,8,8,6,7,8,9,8,9,...,5,6,7,8,9,3,2,1,2,3
4,8,9,9,8,7,8,9,8,6,7,...,7,8,9,9,6,5,3,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,9,5,6,7,8,9,9,8,9,...,8,9,4,2,9,6,7,9,2,3
96,8,9,8,7,9,9,9,8,7,8,...,9,3,2,1,2,5,9,8,9,4
97,9,4,9,9,9,9,8,7,6,7,...,3,2,1,0,3,4,5,7,8,9
98,5,3,2,3,9,8,7,7,5,6,...,4,4,2,1,2,5,6,8,9,5


# Day 10

In [666]:
data10 = pd.read_csv('input10.txt', header=None, sep="\n", dtype=str)

tag_map1 = {
    ')': 3,
    ']': 57,
    '}': 1197,
    '>': 25137,
    '': 0
}
tag_map2 = {
    ')': 1,
    ']': 2,
    '}': 3,
    '>': 4,
}

def remove_closed_tags(inp):
    while True:
        outp = re.sub(r'\(\)|\[\]|\{\}|\<\>', '', inp)
        if outp == inp:
            return outp
        inp = outp

def score2(inp):
    score = 0
    for a in inp:
        score *= 5
        score += tag_map2[a]
    return score

data10m = data10.applymap(remove_closed_tags).applymap(lambda x: (re.findall(r'\)|\]|\}|\>', x) or [''])[0])
res1 = data10m.applymap(lambda x: tag_map1[x])[0].sum()
res2 = data10.applymap(remove_closed_tags)[data10m[0] == ''].applymap(lambda x: x[::-1].replace('(', ')').replace('[', ']').replace('{', '}').replace('<', '>')).applymap(score2)[0].median()

print('ans1: ' + str(res1)) # 318099
print('ans2: ' + str(int(res2))) # 2389738699

ans1: 318099
ans2: 2389738699
